# <font color=red> Image Classification with CNN for Malaria Data

In [ ]:
# Veri/Data = https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria

In [1]:
import pandas as pd
import os #resimleri okuduk
import cv2
import numpy as np
import warnings
warnings.filterwarnings('ignore') 

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Input,MaxPooling2D,Dropout,BatchNormalization,Reshape

In [2]:
labels=['Parasitized','Uninfected'] #labellarımızı tanımladık
img_path='cell_images/'#yukarıdaki tanımladığımız labelları hangi dosyadan çekiceğimizi tanımladık

In [3]:
img_list=[]#okuduğumuz resimlerin isimlerini kaydetmek için liste oluşturduk
label_list=[]#okuduğumuz resimlerin Parasitized ya da Uninfected olup olmadığını yanına yazmak için liste oluşturduk
for label in labels: #labels listesinin içindeki labelları okuduk
    for img_file in os.listdir(img_path+label):# img_file diye bir data dosyası oluşturup içinde img_path+label listdir kodu ile tüm dosya işimlerini listeledik
        img_list.append(img_path+label+'/'+img_file) #tüm dosyaların tam yollarını bir listeye toplayıp en sona .append ile yeni bir eleman ekledik
        label_list.append(label)#.append listeye yeni bir eleman eklemeye yarıyor

In [4]:
df=pd.DataFrame({'img':img_list,'label':label_list})#yeni bir data frame oluşturduk img diye bir sütün oluşturduk img_list verileini bu yeni sütüna ekledik. Aynısını label_list'de yaptık

In [5]:
df.head()

,img,label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized


In [6]:
df.tail()

,img,label
27553,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27554,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27555,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27556,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected
27557,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected


In [7]:
d={'Parasitized':1,'Uninfected':0}#labelları sayısal veriye çevirdik. Ana hedefimiz Parasitized olduğu için en yüksek değeri ona verdik

In [8]:
df['encode_label']=df['label'].map(d)

In [9]:
df.head()

,img,label,encode_label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1


In [10]:
x=[]
for img in df['img']: # img sütunundaki içindeki img verisine bak 
    img=cv2.imread(str(img))#resimleri string olarak okur
    img=cv2.resize(img,(30,30))#Resimleri yeniden boyutlandırdık
    img=img/255.0# piksel değerlerini normalleştirmek için
    x.append(img)

In [11]:
#bunu dışarıya yazdık, çünkü yukarıda yazsaydık dizi olur ve boyut fazlalığı hatası verir
x=np.array(x)#yazıyı resim olarak okuyor, bu yüzden diziye çevirdik

In [12]:
y=df['encode_label']

## <font color='lime'> CNN

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [14]:
model=Sequential()
model.add(Input(shape=(30,30,3))) #İşleme girecek resmin biçimini belirtdik
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(250))
model.add(Dense(150))
model.add(Dense(100))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=15,verbose=1)

Epoch 1/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6150 - loss: 0.6541 - val_accuracy: 0.8628 - val_loss: 0.3293
Epoch 2/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9075 - loss: 0.2489 - val_accuracy: 0.9398 - val_loss: 0.1737
Epoch 3/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9415 - loss: 0.1698 - val_accuracy: 0.9349 - val_loss: 0.1785
Epoch 4/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.9477 - loss: 0.1512 - val_accuracy: 0.9421 - val_loss: 0.1709
Epoch 5/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9459 - loss: 0.1505 - val_accuracy: 0.9287 - val_loss: 0.2057
Epoch 6/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.9488 - loss: 0.1478 - val_accuracy: 0.9434 - val_loss: 0.1671
Epoch 7/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9542 - loss: 0.1330 - val_accuracy: 0.9478 - val_loss: 0.1518
Epoch 8/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.9548 - loss: 0.1329 - 

In [16]:
model.save('My_cnn_model.h5')